In [ ]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 24: Blizzard Basin ---</h2><p>With everything replanted for next year (and with elephants and monkeys to tend the grove), you and the Elves leave for the extraction point.</p>
<p>Partway up the mountain that shields the grove is a flat, open area that serves as the extraction point. It's a bit of a climb, but nothing the expedition can't handle.</p>
<p>At least, that would normally be true; now that the mountain is covered in snow, things have become more difficult than the Elves are used to.</p>
<p>As the expedition reaches a valley that must be traversed to reach the extraction site, you find that strong, turbulent winds are pushing small <em>blizzards</em> of snow and sharp ice around the valley. It's a good thing everyone packed warm clothes! To make it across safely, you'll need to find a way to avoid them.</p>
<p>Fortunately, it's easy to see all of this from the entrance to the valley, so you make a map of the valley and the blizzards (your puzzle input). For example:</p>
<pre><code>#.#####
#.....#
#&gt;....#
#.....#
#...v.#
#.....#
#####.#
</code></pre>
<p>The walls of the valley are drawn as <code>#</code>; everything else is ground. Clear ground - where there is currently no blizzard - is drawn as <code>.</code>. Otherwise, blizzards are drawn with an arrow indicating their direction of motion: up (<code>^</code>), down (<code>v</code>), left (<code>&lt;</code>), or right (<code>&gt;</code>).</p>
<p>The above map includes two blizzards, one moving right (<code>&gt;</code>) and one moving down (<code>v</code>). In one minute, each blizzard moves one position in the direction it is pointing:</p>
<pre><code>
#.#####
#.....#
#.&gt;...#
#.....#
#.....#
#...v.#
#####.#
</code></pre>

<p>Due to <span title="I think, anyway. Do I look like a theoretical blizzacist?">conservation of blizzard energy</span>, as a blizzard reaches the wall of the valley, a new blizzard forms on the opposite side of the valley moving in the same direction. After another minute, the bottom downward-moving blizzard has been replaced with a new downward-moving blizzard at the top of the valley instead:</p>
<pre><code>#.#####
#...v.#
#..&gt;..#
#.....#
#.....#
#.....#
#####.#
</code></pre>
<p>Because blizzards are made of tiny snowflakes, they pass right through each other. After another minute, both blizzards temporarily occupy the same position, marked <code>2</code>:</p>
<pre><code>#.#####
#.....#
#...2.#
#.....#
#.....#
#.....#
#####.#
</code></pre>
<p>After another minute, the situation resolves itself, giving each blizzard back its personal space:</p>
<pre><code>#.#####
#.....#
#....&gt;#
#...v.#
#.....#
#.....#
#####.#
</code></pre>
<p>Finally, after yet another minute, the rightward-facing blizzard on the right is replaced with a new one on the left facing the same direction:</p>
<pre><code>#.#####
#.....#
#&gt;....#
#.....#
#...v.#
#.....#
#####.#
</code></pre>
<p>This process repeats at least as long as you are observing it, but probably forever.</p>
<p>Here is a more complex example:</p>
<pre><code>#.######
#&gt;&gt;.&lt;^&lt;#
#.&lt;..&lt;&lt;#
#&gt;v.&gt;&lt;&gt;#
#&lt;^v^^&gt;#
######.#
</code></pre>
<p>Your expedition begins in the only non-wall position in the top row and needs to reach the only non-wall position in the bottom row. On each minute, you can <em>move</em> up, down, left, or right, or you can <em>wait</em> in place. You and the blizzards act <em>simultaneously</em>, and you cannot share a position with a blizzard.</p>
<p>In the above example, the fastest way to reach your goal requires <code><em>18</em></code> steps. Drawing the position of the expedition as <code>E</code>, one way to achieve this is:</p>
<pre><code>Initial state:
#<em>E</em>######
#&gt;&gt;.&lt;^&lt;#
#.&lt;..&lt;&lt;#
#&gt;v.&gt;&lt;&gt;#
#&lt;^v^^&gt;#
######.#

Minute 1, move down:
#.######<br/>#<em>E</em>&gt;3.&lt;.#
#&lt;..&lt;&lt;.#
#&gt;2.22.#
#&gt;v..^&lt;#
######.#

Minute 2, move down:
#.######
#.2&gt;2..#<br/>#<em>E</em>^22^&lt;#
#.&gt;2.^&gt;#
#.&gt;..&lt;.#
######.#

Minute 3, wait:
#.######
#&lt;^&lt;22.#<br/>#<em>E</em>2&lt;.2.#
#&gt;&lt;2&gt;..#
#..&gt;&lt;..#
######.#

Minute 4, move up:
#.######<br/>#<em>E</em>&lt;..22#
#&lt;&lt;.&lt;..#
#&lt;2.&gt;&gt;.#
#.^22^.#
######.#

Minute 5, move right:
#.######
#2<em>E</em>v.&lt;&gt;#
#&lt;.&lt;..&lt;#
#.^&gt;^22#
#.2..2.#
######.#

Minute 6, move right:
#.######
#&gt;2<em>E</em>&lt;.&lt;#
#.2v^2&lt;#
#&gt;..&gt;2&gt;#
#&lt;....&gt;#
######.#

Minute 7, move down:
#.######
#.22^2.#
#&lt;v<em>E</em>&lt;2.#
#&gt;&gt;v&lt;&gt;.#
#&gt;....&lt;#
######.#

Minute 8, move left:
#.######
#.&lt;&gt;2^.#
#.<em>E</em>&lt;&lt;.&lt;#
#.22..&gt;#
#.2v^2.#
######.#

Minute 9, move up:
#.######
#&lt;<em>E</em>2&gt;&gt;.#
#.&lt;&lt;.&lt;.#
#&gt;2&gt;2^.#
#.v&gt;&lt;^.#
######.#

Minute 10, move right:
#.######
#.2<em>E</em>.&gt;2#
#&lt;2v2^.#
#&lt;&gt;.&gt;2.#
#..&lt;&gt;..#
######.#

Minute 11, wait:
#.######
#2^<em>E</em>^2&gt;#
#&lt;v&lt;.^&lt;#
#..2.&gt;2#
#.&lt;..&gt;.#
######.#

Minute 12, move down:
#.######
#&gt;&gt;.&lt;^&lt;#
#.&lt;<em>E</em>.&lt;&lt;#
#&gt;v.&gt;&lt;&gt;#
#&lt;^v^^&gt;#
######.#

Minute 13, move down:
#.######
#.&gt;3.&lt;.#
#&lt;..&lt;&lt;.#
#&gt;2<em>E</em>22.#
#&gt;v..^&lt;#
######.#

Minute 14, move right:
#.######
#.2&gt;2..#
#.^22^&lt;#
#.&gt;2<em>E</em>^&gt;#
#.&gt;..&lt;.#
######.#

Minute 15, move right:
#.######
#&lt;^&lt;22.#
#.2&lt;.2.#
#&gt;&lt;2&gt;<em>E</em>.#
#..&gt;&lt;..#
######.#

Minute 16, move right:
#.######
#.&lt;..22#
#&lt;&lt;.&lt;..#
#&lt;2.&gt;&gt;<em>E</em>#
#.^22^.#
######.#

Minute 17, move down:
#.######
#2.v.&lt;&gt;#
#&lt;.&lt;..&lt;#
#.^&gt;^22#
#.2..2<em>E</em>#
######.#

Minute 18, move down:
#.######
#&gt;2.&lt;.&lt;#
#.2v^2&lt;#
#&gt;..&gt;2&gt;#
#&lt;....&gt;#<br/>######<em>E</em>#
</code></pre>

<p><em>What is the fewest number of minutes required to avoid the blizzards and reach the goal?</em></p>
</article>


In [ ]:
from copy import deepcopy
from pprint import pprint
from typing import Literal


tests = [
    {
        "name": "Smaller Example",
        "s": """
            #.#####
            #.....#
            #>....#
            #.....#
            #...v.#
            #.....#
            #####.#
       """,
        "expected": 10,
    },
    {
        "name": "Example",
        "s": """
            #.######
            #>>.<^<#
            #.<..<<#
            #>v.><>#
            #<^v^^>#
            ######.#
        """,
        "expected": 18,
    },
]

type State = list[list[list[str]]]


class Valley(Str):
    def __init__(self, s: str) -> None:
        self.state = [
            [[] if c == "." else [c] for c in l.strip()] for l in s.strip().splitlines()
        ]

        self.rows, self.cols = len(self.state), len(self.state[0])

        self.me = 0, 1
        self.goal = self.rows - 1, self.cols - 2

    def next_state(self, state: State, empty: State) -> tuple[State, State]:
        for r, c in product(range(1, self.rows - 1), range(1, self.cols - 1)):
            while state[r][c]:
                blizzard = state[r][c].pop()
                rr, cc = r, c
                if blizzard == "^":
                    rr -= 1
                elif blizzard == "v":
                    rr += 1
                elif blizzard == ">":
                    cc += 1
                elif blizzard == "<":
                    cc -= 1

                rr = 1 + (rr - 1) % (self.rows - 2)
                cc = 1 + (cc - 1) % (self.cols - 2)
                empty[rr][cc].append(blizzard)
        return state, empty

    def hash_state(state: State) -> int:
        return hash(str(state))

    def min_minutes_to_goal(self) -> int:
        state, empty, me, goal = self.init_state()
        print(goal)
        print(self.state_str(state))

        for _ in range(5):
            empty, state = self.next_state(state, empty)
            print()
            print(self.state_str(state))

        return -1

    def init_state(self) -> tuple[State, State, tuple[int, int], tuple[int, int]]:
        state = deepcopy(self.state)
        empty = [[[] for _ in range(self.cols)] for _ in range(self.rows)]

        for r in range(self.rows):
            empty[r][0].append("#")
            empty[r][-1].append("#")

        for c in range(self.cols):
            empty[0][c].append("#")
            empty[-1][c].append("#")

        return state, empty, self.me, self.goal

    @classmethod
    def state_str(cls, state: State, me: tuple[int, int] = None) -> str:
        rows, cols = len(state), len(state[0])
        me = (0, 1) if me is None else me

        str_buffer = [["."] * cols for _ in range(rows)]

        for r, c in product(range(rows), range(cols)):
            if (r, c) == me:
                str_buffer[r][c] = "E"
            elif len(state[r][c]) == 1:
                str_buffer[r][c] = state[r][c][0]
            elif len(state[r][c]) > 1:
                str_buffer[r][c] = str(len(state[r][c]))

        return "\n".join(
            (
                f"{rows=},{cols=}",
                "\n".join("".join(row) for row in str_buffer),
            )
        )


@test(tests=tests)
def test_part_I(s: str) -> int:
    return Valley(s).min_minutes_to_goal()

In [ ]:
with open("../input/day24.txt") as f:
    puzzle = f.read()

# print(f"Part I: {Scan.empty_ground_tiles(puzzle, 10)}")
print(Valley(puzzle).min_minutes_to_goal())

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3906</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2>


In [ ]:
# from collections import frozenmap

In [ ]:
# print(f"PartI II: {ScanII.first_round_no_elfs_moved(puzzle)}")

<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>895</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
